In [2]:
import pandas as pd

In [ ]:
!pip list

In [3]:
!pip install pyarrow --quiet

In [16]:

train_inputs = pd.read_parquet('train_inputs.parquet')
val_inputs = pd.read_parquet('val_inputs.parquet')
test_inputs=pd.read_parquet('test_inputs.parquet')

train_targets = pd.read_parquet('train_targets.parquet')
val_targets = pd.read_parquet('val_targets.parquet')


In [ ]:
train_inputs.head()

,Age,Occupation,Annual_Income,Monthly_Inhand_Salary,Num_Bank_Accounts,Num_Credit_Card,Interest_Rate,Num_of_Loan,Delay_from_due_date,Num_of_Delayed_Payment,...,Credit_Mix_Good,Credit_Mix_Standard,Payment_of_Min_Amount_No,Payment_of_Min_Amount_Yes,Payment_Behaviour_High_spent_Large_value_payments,Payment_Behaviour_High_spent_Medium_value_payments,Payment_Behaviour_High_spent_Small_value_payments,Payment_Behaviour_Low_spent_Large_value_payments,Payment_Behaviour_Low_spent_Medium_value_payments,Payment_Behaviour_Low_spent_Small_value_payments
75220,0.056534,Journalist,0.003902,0.552466,0.001112,0.002001,0.000173,0.063910,0.263889,0.002045,...,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
48955,0.057078,Accountant,0.004377,0.629572,0.003335,0.000667,0.001553,0.065163,0.236111,0.002955,...,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
44966,0.059687,Scientist,0.000872,0.125483,0.003335,0.004670,0.001208,0.064536,0.458333,0.002500,...,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
13568,0.058056,Manager,0.000638,0.104235,0.004447,0.004670,0.001380,0.065163,0.291667,0.003636,...,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
92727,0.057404,Media_Manager,0.000577,0.110080,0.001668,0.002001,0.001380,0.063283,0.097222,0.002727,...,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0


In [ ]:
val_inputs.info()

<class 'pandas.core.frame.DataFrame'>
Index: 20000 entries, 75721 to 42410
Data columns (total 46 columns):
 #   Column                                              Non-Null Count  Dtype  
---  ------                                              --------------  -----  
 0   Age                                                 20000 non-null  float64
 1   Occupation                                          20000 non-null  object 
 2   Annual_Income                                       20000 non-null  float64
 3   Monthly_Inhand_Salary                               20000 non-null  float64
 4   Num_Bank_Accounts                                   20000 non-null  float64
 5   Num_Credit_Card                                     20000 non-null  float64
 6   Interest_Rate                                       20000 non-null  float64
 7   Num_of_Loan                                         20000 non-null  float64
 8   Delay_from_due_date                                 20000 non-null  float64
 

In [17]:
test_inputs.shape

(50000, 46)

In [ ]:
train_targets.info()

<class 'pandas.core.frame.DataFrame'>
Index: 80000 entries, 75220 to 15795
Data columns (total 1 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Credit_Score  80000 non-null  object
dtypes: object(1)
memory usage: 1.2+ MB


In [ ]:
train_targets['Credit_Score']

75220    Standard
48955        Good
44966        Good
13568    Standard
92727        Good
           ...   
6265         Good
54886        Poor
76820    Standard
860      Standard
15795        Poor
Name: Credit_Score, Length: 80000, dtype: object

In [6]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from  sklearn.ensemble import GradientBoostingClassifier ,RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

In [18]:
import numpy as np

def trainModel(Model, metric):
    numeric_cols = train_inputs.select_dtypes(include=np.number).columns.tolist()
    X_train = train_inputs[numeric_cols]
    X_val = val_inputs[numeric_cols]
    X_test = test_inputs[numeric_cols]
    y_train = train_targets.values.ravel()
    y_val = val_targets.values.ravel()

    model = Model
    model.fit(X_train, y_train)

    train_preds = model.predict(X_train)
    print(f"Training evaluation: {metric(y_train, train_preds) * 100:.2f}%")

    val_preds = model.predict(X_val)
    print(f"Validation evaluation: {metric(y_val, val_preds) * 100:.2f}%")

    test_preds = model.predict(X_test)
    print("Test predictions:")
    print(test_preds)


In [19]:

model = LogisticRegression(
    penalty='l2',
    dual=False,
    tol=0.0001,
    C=1.0,
    fit_intercept=True,
    intercept_scaling=1,
    class_weight=None,
    random_state=None,
    solver='saga',
    max_iter=100,
    multi_class='multinomial',
    verbose=0,
    warm_start=False,
    n_jobs=None,
    l1_ratio=None
)

metric=accuracy_score
trainModel(model,metric)

Training evaluation: 61.81%
Validation evaluation: 61.58%
Test predictions:
['Good' 'Good' 'Good' ... 'Standard' 'Standard' 'Standard']


In [20]:
model = DecisionTreeClassifier(
    criterion='gini',
    splitter='best',
    max_depth=15,
    min_samples_split=10,
    min_samples_leaf=5,
    min_weight_fraction_leaf=0.0,
    max_features=None,
    random_state=None,
    max_leaf_nodes=None,
    min_impurity_decrease=0.0,
    class_weight=None,
    ccp_alpha=0.0
)
metric = accuracy_score
trainModel(model, metric)

Training evaluation: 79.15%
Validation evaluation: 70.95%
Test predictions:
['Good' 'Good' 'Good' ... 'Standard' 'Standard' 'Standard']


In [21]:
model = GaussianNB(
    priors=None,
    var_smoothing=1e-9
)
metric=accuracy_score
trainModel(model,metric)

Training evaluation: 57.82%
Validation evaluation: 57.64%
Test predictions:
['Good' 'Good' 'Good' ... 'Good' 'Good' 'Standard']


In [22]:
model = GradientBoostingClassifier(
    loss='log_loss',
    learning_rate=0.1,
    n_estimators=300,
    subsample=0.8,
    criterion='friedman_mse',
    min_samples_split=2,
    min_samples_leaf=1,
    min_weight_fraction_leaf=0.0,
    max_depth=3,
    min_impurity_decrease=0.0,
    init=None,
    random_state=42,
    max_features=None,
    verbose=0,
    max_leaf_nodes=None,
    warm_start=False,
    validation_fraction=0.1,
    n_iter_no_change=None,
    tol=0.0001,
    ccp_alpha=0.0
)
metric = accuracy_score
trainModel(model, metric)



Training evaluation: 73.17%
Validation evaluation: 71.34%
Test predictions:
['Good' 'Good' 'Good' ... 'Poor' 'Standard' 'Standard']


In [23]:
model = RandomForestClassifier(
    n_estimators=500,
    criterion='entropy',
    max_depth=25,
    min_samples_split=2,
    min_samples_leaf=1,
    random_state=42,
    ccp_alpha=0.001
)
metric=accuracy_score
trainModel(model,metric)

Training evaluation: 68.91%
Validation evaluation: 68.48%
Test predictions:
['Good' 'Good' 'Good' ... 'Poor' 'Good' 'Standard']


In [24]:
model = KNeighborsClassifier(
    n_neighbors=10,
    weights='uniform',
    algorithm='auto',
    leaf_size=30,
    p=2,
    metric='minkowski',
    n_jobs=None
)
metric = accuracy_score
trainModel(model, metric)


Training evaluation: 70.10%
Validation evaluation: 64.22%
Test predictions:
['Good' 'Standard' 'Standard' ... 'Poor' 'Standard' 'Standard']
